## XAFS Data Processing with wxPython Plotting

Here we'll use Larch for XAFS data processing:

  1. read data from an Athena Project File
  2. do XAFS background subtraction with autobk()
  3. do XAFS Fourier transform with xfft()

For more details, see https://xraypy.github.io/xraylarch/xafs/

Larch's normal plotting uses wxPython.  This can be used with Jupyter, displaying plots in an interactive but separate window from the browser.   This only works when running Jupyter on a local machine. If running on Jupyter hub or binder-like site, you should either use the larch plotly plotting tools or use standard matplotlib.

To use Larch's wxPython functionality, you'll need to do

%gui wx

before doing any plotting

We'll start with reading and exploring the Athena Project File:

In [11]:
%gui wx
from larch.wxlib import _plot as plot
from larch.wxlib.xafsplots import plot_mu, plot_bkg, plot_chik, plot_chir

from larch.io import read_athena
project = read_athena('../xafsdata/fe_athena.prj')

In [12]:
for name, group in project._athena_groups.items():
    print(name, group)

fe2o3_rt1_xmu <Group 0x19a9c1010>
fe3c_rt_xdi <Group 0x1998497d0>
feo_rt1_xmu <Group 0x199849c50>


> **NOTE** For additional features of working with Athena project files -> [IO_Athena_project.ipynb](./IO_Athena_project.ipynb) 

Let's get that data group for Fe2O3 and look at what that Group object contains

In [13]:
fe2o3 = project.fe2o3_rt1_xmu
fe2o3.filename = 'Fe2O3'
for attr in dir(fe2o3):
    print(attr, type(getattr(fe2o3, attr)))

athena_id <class 'str'>
athena_params <class 'larch.symboltable.Group'>
atsym <class 'str'>
callargs <class 'larch.symboltable.Group'>
d2mude <class 'numpy.ndarray'>
dmude <class 'numpy.ndarray'>
e0 <class 'numpy.float64'>
edge <class 'str'>
edge_step <class 'float'>
edge_step_poly <class 'float'>
energy <class 'numpy.ndarray'>
epsk <class 'float'>
epsr <class 'float'>
filename <class 'str'>
flat <class 'numpy.ndarray'>
i0 <class 'numpy.ndarray'>
journal <class 'larch.larchlib.Journal'>
label <class 'str'>
mu <class 'numpy.ndarray'>
norm <class 'numpy.ndarray'>
norm_poly <class 'numpy.ndarray'>
post_edge <class 'numpy.ndarray'>
pre_edge <class 'numpy.ndarray'>
pre_edge_details <class 'larch.symboltable.Group'>
sel <class 'int'>
signal <class 'numpy.ndarray'>


There are several components to this Group of data.  We note that there are arrays for `energy` and `mu`.   These are what represent $\mu(E)$.

We'll want to plot this data and the other data arrays as we go.  

In [14]:
plot(fe2o3.energy, fe2o3.mu, xlabel='E (eV)', ylabel=r'$\mu(E)$', new=True)
# or plot_mu(fe2o3, new=True)

Now let's remove the XAFS background and extract the EXAFS $\chi(k)$.   We'll use
the `autobk()` function (https://xraypy.github.io/xraylarch/xafs/autobk.html)

Note that we pass in the *Group* as the first argument.  The `autobk()` function will add several attributes to this Group -- it will be the container for this dataset.

In [15]:
from larch.xafs import autobk
autobk(fe2o3, rbkg=0.85, kweight=2)

plot_chik(fe2o3, kweight=2)

we can also plot the background $\mu_0(E)$:

In [16]:
plot_bkg(fe2o3, win=2)

and now we will do the Fourier transform from $\chi(k)$ to $\chi(R)$, using the `xftf()` (XAFS Fourier Transform Forward) function.  We'll do the Fourier transform on $k^2$ weighted $\chi(k)$ with a Kaiser-Bessel Window. 

In [17]:
from larch.xafs import xftf
xftf(fe2o3, kweight=2, kmin=2, kmax=13.0, dk=5, kwindow='Kaiser-Bessel')

In [9]:
plot_chik(fe2o3, kweight=2, show_window=True, new=True)

and we can show that data as $\chi(R)$

In [10]:
plot_chir(fe2o3, show_mag=True, show_real=True, win=2)
